In [1]:
from __future__ import division
import sys
sys.path.append('./../cbp')
import utils
import cbp
import pandas as pd
import secrets
from cbp import Counties

In [2]:
# Example code for downloading from Census API
# texas_api = Counties(state_fips='48', read_from='api', key=secrets.censuskey)
# texas_api.head()
# texas_api.to_csv('texas_cbp_2014.csv', index=False)

In [3]:
# Instantiate Counties object for Texas 2014
texas = Counties(state_fips='48', read_from='csv', filepath='texas_cbp_2014.csv')
texas.head()

,EMP,ESTAB,NAICS2012,NAICS2012_TTL,GEO_TTL,state,county
0,11738,940,00,Total for all sectors,Anderson County,48,001
1,5683,403,00,Total for all sectors,Andrews County,48,003
2,30525,1857,00,Total for all sectors,Angelina County,48,005
3,4131,503,00,Total for all sectors,Aransas County,48,007
4,1539,201,00,Total for all sectors,Archer County,48,009


In [4]:
# Identify FIPS codes for our five Austin-Round Rock MSA counties from the data

fips = []
for county in "Bastrop, Caldwell, Hays, Travis, Williamson".split(", "):
    fips.append(texas[(texas.NAICS2012 == '00') & (texas.GEO_TTL.str.contains(county))]['county'].iloc[0])
print fips

['021', '055', '209', '453', '491']


In [5]:
# Get data at the two-digit NAICS level, for only the Austin MSA counties
austin_region = texas.three_digit(county=fips)
austin_region.head()

,EMP,ESTAB,NAICS2012,NAICS2012_TTL,GEO_TTL,state,county
10,12475,1175,00,Total for all sectors,Bastrop County,48,021
27,6116,580,00,Total for all sectors,Caldwell County,48,055
104,46675,3660,00,Total for all sectors,Hays County,48,209
225,544038,32217,00,Total for all sectors,Travis County,48,453
244,136393,9252,00,Total for all sectors,Williamson County,48,491


In [6]:
austin_all = texas.get_county(fips)

In [7]:
# Use groupby to aggregate data into MSA-level totals

aggs = {
    'EMP':'sum',
    'ESTAB':'sum',
    'NAICS2012_TTL':'first'
}
austin_msa = austin_region.groupby('NAICS2012',as_index=False).agg(aggs)
austin_msa['GEO_TTL'] = 'Austin-Round Rock MSA'
austin_msa.head()

,NAICS2012,NAICS2012_TTL,ESTAB,EMP,GEO_TTL
0,00,Total for all sectors,46884,745697,Austin-Round Rock MSA
1,114,"Fishing, Hunting and Trapping",1,10,Austin-Round Rock MSA
2,115,Support Activities for Agriculture and Forestry,27,60,Austin-Round Rock MSA
3,211,Oil and Gas Extraction,95,607,Austin-Round Rock MSA
4,212,Mining (except Oil and Gas),30,1051,Austin-Round Rock MSA


In [8]:
# Use groupby to aggregate data into MSA-level totals

aggs = {
    'EMP':'sum',
    'ESTAB':'sum',
    'NAICS2012_TTL':'first'
}
austin_msa_all = austin_all.groupby('NAICS2012',as_index=False).agg(aggs)
austin_msa_all['GEO_TTL'] = 'Austin-Round Rock MSA'
austin_msa_all.head()

,NAICS2012,NAICS2012_TTL,ESTAB,EMP,GEO_TTL
0,00,Total for all sectors,46884,745697,Austin-Round Rock MSA
1,11,"Agriculture, Forestry, Fishing and Hunting",28,61,Austin-Round Rock MSA
2,114,"Fishing, Hunting and Trapping",1,10,Austin-Round Rock MSA
3,1142,Hunting and Trapping,1,10,Austin-Round Rock MSA
4,11421,Hunting and Trapping,1,10,Austin-Round Rock MSA


In [9]:
# LOCATION QUOTIENT

# Create series for employment by sector for Texas
texas_total_emp = texas.three_digit().groupby('NAICS2012').agg('sum').EMP
# Create series for employment by sector for Austin
austin_msa_emp = austin_msa.set_index('NAICS2012').EMP

# Use utils.location_quotient to calculate LQ
lq = pd.DataFrame(utils.location_quotient(austin_msa_emp, texas_total_emp))

# Merge LQ results into our DataFrame
austin_msa = austin_msa.merge(lq, left_on='NAICS2012', right_index=True)

In [10]:
# View results
austin_msa.sort_values('location_quotient', ascending=False)

,NAICS2012,NAICS2012_TTL,ESTAB,EMP,GEO_TTL,location_quotient
60,519,Other Information Services,125,2145,Austin-Round Rock MSA,2.807632
55,511,Publishing Industries (except Internet),270,9213,Austin-Round Rock MSA,2.735200
26,334,Computer and Electronic Product Manufacturing,136,14705,Austin-Round Rock MSA,2.500175
59,518,"Data Processing, Hosting, and Related Services",160,9572,Austin-Round Rock MSA,2.261087
49,485,Transit and Ground Passenger Transportation,62,3310,Austin-Round Rock MSA,2.042096
45,454,Nonstore Retailers,468,4826,Austin-Round Rock MSA,1.777115
67,533,Lessors of Nonfinancial Intangible Assets (exc...,24,551,Austin-Round Rock MSA,1.776840
56,512,Motion Picture and Sound Recording Industries,214,2518,Austin-Round Rock MSA,1.635953
68,541,"Professional, Scientific, and Technical Services",7984,78996,Austin-Round Rock MSA,1.554784
33,425,Wholesale Electronic Markets and Agents and Br...,222,2796,Austin-Round Rock MSA,1.454637


In [11]:
lq = pd.DataFrame(utils.location_quotient(austin_msa_all.set_index('NAICS2012').EMP,
                                          texas.groupby('NAICS2012').agg('sum').EMP))

# Merge LQ results into our DataFrame
austin_msa_all = austin_msa_all.merge(lq, left_on='NAICS2012', right_index=True)

In [12]:
# Clusters
movies = austin_msa_all[austin_msa_all.NAICS2012.str.startswith('512') | 
                        austin_msa_all.NAICS2012.str.startswith('515') |
                        austin_msa_all.NAICS2012.str.startswith('517')]

In [13]:
movies[(movies.location_quotient > 1) & (movies.NAICS2012.str.len() == 5)]

,NAICS2012,NAICS2012_TTL,ESTAB,EMP,GEO_TTL,location_quotient
1077,51211,Motion Picture and Video Production,124,527,Austin-Round Rock MSA,2.817354
1081,51213,Motion Picture and Video Exhibition,37,1807,Austin-Round Rock MSA,1.475074
1084,51219,Postproduction Services and Other Motion Pictu...,26,97,Austin-Round Rock MSA,1.211157
1088,51221,Record Production,1,10,Austin-Round Rock MSA,2.574644
1094,51224,Sound Recording Studios,18,80,Austin-Round Rock MSA,2.340586
1096,51229,Other Sound Recording Industries,3,20,Austin-Round Rock MSA,2.076326
1103,51512,Television Broadcasting,11,768,Austin-Round Rock MSA,1.194326
1107,51711,Wired Telecommunications Carriers,193,5801,Austin-Round Rock MSA,1.064071
1116,51791,Other Telecommunications,39,400,Austin-Round Rock MSA,1.096527


In [21]:
computers = austin_msa_all[austin_msa_all.NAICS2012.str.startswith('334')]

In [23]:
# computers[(computers.location_quotient > 1) & (computers.NAICS2012.str.len() == 5)]
computers[(computers.NAICS2012.str.len() == 5)]

,NAICS2012,NAICS2012_TTL,ESTAB,EMP,GEO_TTL,location_quotient
482,33411,Computer and Peripheral Equipment Manufacturing,13,343,Austin-Round Rock MSA,1.934085
486,33421,Telephone Apparatus Manufacturing,5,1760,Austin-Round Rock MSA,5.109804
488,33422,Radio and Television Broadcasting and Wireless...,3,70,Austin-Round Rock MSA,0.306714
490,33429,Other Communications Equipment Manufacturing,5,235,Austin-Round Rock MSA,3.372583
493,33431,Audio and Video Equipment Manufacturing,4,30,Austin-Round Rock MSA,1.119411
496,33441,Semiconductor and Other Electronic Component M...,53,10170,Austin-Round Rock MSA,3.457199
503,33451,"Navigational, Measuring, Electromedical, and C...",49,2518,Austin-Round Rock MSA,0.907291
513,33461,Manufacturing and Reproducing Magnetic and Opt...,4,20,Austin-Round Rock MSA,0.705382


In [18]:
austin_msa.to_csv('austin_msa_2014_threedigit_LQ.csv')

In [10]:
# Get four digit

fourdigit = texas.four_digit(county=fips)

# Use groupby to aggregate data into MSA-level totals

aggs = {
    'EMP':'sum',
    'ESTAB':'sum',
    'NAICS2012_TTL':'first'
}
austin_msa_4 = fourdigit.groupby('NAICS2012',as_index=False).agg(aggs)
austin_msa_4['GEO_TTL'] = 'Austin-Round Rock MSA'
austin_msa_4.head()

# Use utils.location_quotient to calculate LQ
lq = pd.DataFrame(utils.location_quotient(austin_msa_4.set_index('NAICS2012').EMP,
                                          texas.four_digit().groupby('NAICS2012').agg('sum').EMP))

austin_msa_4 = austin_msa_4.merge(lq, left_on='NAICS2012', right_index=True)
austin_msa_4.head()

,NAICS2012,NAICS2012_TTL,ESTAB,EMP,GEO_TTL,location_quotient
0,00,Total for all sectors,46884,745697,Austin-Round Rock MSA,1.000000
1,1142,Hunting and Trapping,1,10,Austin-Round Rock MSA,0.286072
2,1151,Support Activities for Crop Production,7,40,Austin-Round Rock MSA,0.092248
3,1152,Support Activities for Animal Production,19,52,Austin-Round Rock MSA,0.325271
4,1153,Support Activities for Forestry,1,10,Austin-Round Rock MSA,0.401035


In [11]:
austin_msa_4.to_csv('./austin_msa_2014_fourdigit_LQ.csv')

In [5]:
austin_msa_4.sort_values('EMP', ascending=False)

,NAICS2012,NAICS2012_TTL,ESTAB,EMP,GEO_TTL
0,00,Total for all sectors,46884,745697,Austin-Round Rock MSA
261,7225,Restaurants and Other Eating Places,3194,75341,Austin-Round Rock MSA
217,5617,Services to Buildings and Dwellings,952,24620,Austin-Round Rock MSA
213,5613,Employment Services,351,18524,Austin-Round Rock MSA
205,5415,Computer Systems Design and Related Services,1705,17792,Austin-Round Rock MSA
210,5511,Management of Companies and Enterprises,346,15608,Austin-Round Rock MSA
229,6211,Offices of Physicians,1336,15303,Austin-Round Rock MSA
203,5413,"Architectural, Engineering, and Related Services",1099,14682,Austin-Round Rock MSA
19,2382,Building Equipment Contractors,991,13804,Austin-Round Rock MSA
128,4451,Grocery Stores,402,12825,Austin-Round Rock MSA


In [5]:
fourdigit.head()

,EMP,ESTAB,NAICS2012,NAICS2012_TTL,GEO_TTL,state,county


In [19]:
# Get 2006 data for change-over-time analysis
texas06 = Counties(state_fips='48', year=2006, read_from='csv', filepath='texas_cbp_2006.csv')

# Get Austin counties and aggregate to MSA
austin_region_06 = texas06.three_digit(county=fips)
austin_msa_06 = austin_region_06.groupby('NAICS2012',as_index=False).agg('sum')
austin_msa_06['GEO_TTL'] = 'Austin-Round Rock MSA'

In [20]:
# Clean up Austin 2006 DataFrame
austin_msa_06 = austin_msa_06.merge(austin_msa[['NAICS2012', 'NAICS2012_TTL']], on='NAICS2012')
austin_msa_06.drop('old_naics', axis=1, inplace=True)

In [21]:
austin_msa_06.head()

,NAICS2012,Unnamed: 0,EMP,ESTAB,GEO_TTL,NAICS2012_TTL
0,00,331581,595807,36026,Austin-Round Rock MSA,Total for all sectors
1,114,128035,10,1,Austin-Round Rock MSA,"Fishing, Hunting and Trapping"
2,115,331591,80,19,Austin-Round Rock MSA,Support Activities for Agriculture and Forestry
3,211,331613,260,69,Austin-Round Rock MSA,Oil and Gas Extraction
4,212,331626,832,28,Austin-Round Rock MSA,Mining (except Oil and Gas)


In [22]:
subset06 = austin_msa_06[['NAICS2012', 'EMP', 'ESTAB']].rename(columns={'EMP':'EMP2006', 'ESTAB':'ESTAB2006'})
merged = austin_msa.merge(subset06, on='NAICS2012')

In [23]:
merged.to_csv('austin_msa_2006_2014_threedigit.csv')

In [12]:
# Get 2006 series for MSA and Texas employment by sector
austin_msa_emp_06 = austin_msa_06.set_index('NAICS2012').EMP
texas_total_emp_06 = texas06.two_digit().groupby('NAICS2012').agg('sum').EMP

In [13]:
# Location Quotient for 2006
lq_06 = pd.DataFrame(utils.location_quotient(austin_msa_emp_06, texas_total_emp_06))
austin_msa_06 = austin_msa_06.merge(lq_06, left_on='NAICS2012', right_index=True)

In [14]:
# Pass the 4 series (2014 and 2006 employment for Texas and for Austin MSA) to shift share function from utils
shift_share, shift_share_summary = utils.shift_share(small_old=austin_msa_emp_06,
                                                     small_new=austin_msa_emp,
                                                     large_old=texas_total_emp_06,
                                                     large_new=texas_total_emp)

In [15]:
# View detailed results by sector
shift_share

,small_old,small_new,large_old,large_new,large_growth_share,large_industry_growth_rate,industry_mix,small_industry_growth_rate,local_competitiveness
NAICS2012,,,,,,,,,
00,609646,745697,8711361,9598578,62089.987452,0.101846,0.000000,0.223164,73961.012548
11,55,21,5343,3902,5.601528,-0.269699,-20.434955,-0.618182,-19.166573
21,1439,2440,141126,195803,146.556349,0.387434,410.961068,0.695622,443.482583
22,789,2282,28784,35955,80.356469,0.249131,116.208254,1.892269,1296.435277
23,41128,45334,551643,581674,4188.721002,0.054439,-1949.745853,0.102266,1967.024851
31-33,50637,42592,835966,772668,5157.174319,-0.075718,-8991.326456,-0.158876,-4210.847862
42,39017,34186,462570,496565,3973.724162,0.073492,-1106.303199,-0.123818,-7698.420963
44-45,82717,98831,1127808,1239705,8424.392995,0.099216,-217.513678,0.194809,7907.120683
48-49,11020,13551,349109,395123,1122.342575,0.131804,330.138684,0.229673,1078.518741


In [16]:
# View summary
shift_share_summary

,description,absolute,percentage
small_growth,Growth in smaller geography,136761.000000,0.227547
large_growth,Growth in larger geography,784017.000000,0.090705
large_growth_share,Growth attributable to larger geography growth...,61211.667818,0.447581
industry_mix,Growth attributable to industry mix,-7224.713613,-0.052827
local_competitiveness,Growth attributable to local competitiveness,82774.045795,0.605246


In [17]:
# Coefficient of Specialization

# Use utils.location_quotient to calculate LQ
cs_sectors, cs = utils.specialization_coefficient(austin_msa_emp, texas_total_emp)
cs_sectors_06, cs_06 = utils.specialization_coefficient(austin_msa_emp_06, texas_total_emp_06)

In [18]:
print cs_06, cs

0.0966522557497 0.111936597686


In [19]:
austin_msa.to_csv('austin_msa_2014.csv', index=False)
austin_msa_06.to_csv('austin_msa_2006.csv', index=False)
cs_sectors.to_csv('specialization_2014.csv')
cs_sectors_06.to_csv('specialization_2006.csv')
shift_share.to_csv('shift_share_2006_2014.csv')